In [4]:
from langchain_groq import ChatGroq
llm = ChatGroq(groq_api_key = "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",model_name = "llama-3.1-70b-versatile")

In [1]:
from langchain import PromptTemplate
import psycopg2
from psycopg2 import sql
from langchain.tools import tool

@tool("generate_scenario")
def generate_scenario(user_id):
    """
    Generates a thought-provoking but relatable scenario where the user is speaking at a public event
    and stores it in the user's tasks table as an uncompleted task. The task is associated with the user_id, an integer.

    Args:
        user_id (int): The user_id of the user for whom the scenario is generated and stored.
    
    Returns:
        str: A detailed scenario for public speaking based on the user's preferences.
    """
    db_config = {
    "user": "postgres.gxhqfynolyjyzscmddmi",
    "password": "1234",
    "host": "aws-0-ap-south-1.pooler.supabase.com",
    "port": "6543",
    "dbname": "postgres"
}

    prompt = '''
    You are an expert event planner specializing in creating engaging public speaking scenarios. 
    Your task is to craft a realistic, thought-provoking scenario where the user is invited to speak 
    at a public event. The event should be focused on a meaningful yet everyday topic, relevant to 
    the real world. Describe the scenario in vivid detail, including:

    1. The type of event (e.g., a community gathering, a corporate seminar, a university lecture).
    2. The audience composition (e.g., professionals, students, general public).
    3. The topic of the speech, ensuring it is relevant to the event's theme and thought-provoking.
    4. The atmosphere and setting of the event, including details like the venue and mood.
    5. Any challenges or unique aspects the speaker might face, such as audience questions or time constraints.

    Ensure the scenario is engaging and encourages the user to reflect on the importance of their role 
    as a speaker. Keep the description straightforward and relatable.
    '''
    
    connection = None
    cursor = None

    try:
        response = llm.invoke(prompt)
        print(response)
        connection = psycopg2.connect(**db_config)
        cursor = connection.cursor()
        # Insert the scenario into the user's tasks table
        insert_query = sql.SQL(
            "INSERT INTO tasks (user_id, task, completed, score) VALUES (%s, %s, %s, %s)"
        )
        cursor.execute(insert_query, (user_id, response.content, False, None))
        connection.commit()

        return f"Scenario successfully generated {response.content}"

    except psycopg2.Error as e:
        return f"Database error: {e}"
    except Exception as e:
        return f"Error generating scenario: {e}"
    finally:
        if cursor:
            cursor.close()
        if connection:
            connection.close()


In [6]:
import psycopg2
import os

# Database configuration
db_config = {
    "user": os.getenv("DB_USER"),
    "password": os.getenv("DB_PASSWORD"),
    "host": os.getenv("DB_HOST"),
    "port": os.getenv("DB_PORT"),
    "dbname": os.getenv("DB_NAME"),
}

try:
    # Establish the connection
    connection = psycopg2.connect(**db_config)
    cursor = connection.cursor()

    # Insert a dummy user into the users table
    insert_user_query = """
    INSERT INTO users (username, email, password)
    VALUES (%s, %s, %s) RETURNING user_id;
    """
    dummy_user = ("dummyuser", "dummyuser@example.com", "securepassword123")

    # Execute the query
    cursor.execute(insert_user_query, dummy_user)

    # Fetch the inserted user_id
    user_id = cursor.fetchone()[0]
    print(f"Dummy user inserted with user_id: {user_id}")

    # Commit the changes
    connection.commit()

except Exception as e:
    print("An error occurred:", e)

finally:
    if connection:
        cursor.close()
        connection.close()
        print("Database connection closed.")


Dummy user inserted with user_id: 1
Database connection closed.


In [5]:
res = generate_scenario({'user_id': '1'})
res

content='**Event Scenario: "Sustainability in Daily Life" Community Forum**\n\nYou are invited to speak at a community forum focused on exploring the practical ways individuals can contribute to a more sustainable future. The event, titled "Sustainability in Daily Life," aims to inspire and educate the local community on making environmentally conscious choices in their everyday routines.\n\n**Event Type and Audience:**\nThe event is a community gathering, open to the general public, with an expected audience of approximately 200 attendees. The audience composition is diverse, including professionals, students, families, and retirees, all united by their interest in sustainability and environmental conservation.\n\n**Topic of the Speech:**\nYour speech is titled "Small Actions, Big Impact: How Individual Choices Can Drive Community-Wide Sustainability." You will discuss the significance of personal responsibility in environmental conservation, highlighting simple yet effective changes 

'Database error: insert or update on table "tasks" violates foreign key constraint "tasks_user_id_fkey"\nDETAIL:  Key (user_id)=(1) is not present in table "users".\n'

In [32]:
from langchain.tools import tool

@tool("review_speech")
def review_speech(scenario):
    """
    Reviews a user's speech video based on the provided scenario. The function evaluates the content, 
    pronunciation, and facial expressions of the user and provides actionable feedback on areas for improvement. 
    Additionally, it scores each category strictly out of 10, where a 10 represents near perfection.

    Args:
        scenario (str): The scenario or context for the speech.
    Returns:
        dict: A detailed review with suggestions and scores for content, pronunciation, and facial expressions.
    """
    from langchain_core.prompts import PromptTemplate

    template = '''
    You are a professional speech coach with expertise in public speaking, communication skills, 
    and body language analysis. Your task is to review the user's speech performance based on the given scenario 
    and the content of their speech video. Provide detailed feedback in the following areas:

    1. **Content**: Evaluate the relevance, structure, and impact of the speech content. Highlight strengths 
       and suggest improvements for clarity, engagement, and alignment with the scenario's topic.

    2. **Pronunciation**: Assess the user's pronunciation, enunciation, and fluency. Identify areas where the 
       user could improve to make their delivery clearer and more professional.

    3. **Facial Expressions**: Review the user's facial expressions and non-verbal cues. Suggest ways to make 
       their expressions more engaging, appropriate, and aligned with the speech's tone.

    Provide specific examples and actionable suggestions for each category. Then, strictly score the user's 
    performance in each area out of 10, where 10 represents near perfection, 7-9 represents excellent with minor 
    improvements needed, 5-6 represents average with noticeable room for improvement, and below 5 represents 
    significant issues requiring attention.

    Scenario: {scenario}

    Provide the feedback and scores in this structured format:
    - Content Feedback:
      [Detailed feedback]
    - Pronunciation Feedback:
      [Detailed feedback]
    - Facial Expressions Feedback:
      [Detailed feedback]
    - Scores:
      - Content: [Score out of 10]
      - Pronunciation: [Score out of 10]
      - Facial Expressions: [Score out of 10]
    '''

    prompt = PromptTemplate(
        input_variables=['scenario'],
        template=template
    )
    
    response = llm(prompt.format(scenario=scenario))
    return response


In [33]:
from langchain_core.messages import ToolMessage
from langchain_core.runnables import RunnableLambda

from langgraph.prebuilt import ToolNode


def handle_tool_error(state) -> dict:
    error = state.get("error")
    tool_calls = state["messages"][-1].tool_calls
    return {
        "messages": [
            ToolMessage(
                content=f"Error: {repr(error)}\n please fix your mistakes.",
                tool_call_id=tc["id"],
            )
            for tc in tool_calls
        ]
    }


def create_tool_node_with_fallback(tools: list) -> dict:
    return ToolNode(tools).with_fallbacks(
        [RunnableLambda(handle_tool_error)], exception_key="error"
    )


def _print_event(event: dict, _printed: set, max_length=1500):
    current_state = event.get("dialog_state")
    if current_state:
        print("Currently in: ", current_state[-1])
    message = event.get("messages")
    if message:
        if isinstance(message, list):
            message = message[-1]
        if message.id not in _printed:
            msg_repr = message.pretty_repr(html=True)
            if len(msg_repr) > max_length:
                msg_repr = msg_repr[:max_length] + " ... (truncated)"
            print(msg_repr)
            _printed.add(message.id)

In [34]:
from typing import Annotated

from typing_extensions import TypedDict

from langgraph.graph.message import AnyMessage, add_messages


class State(TypedDict):
    messages: Annotated[list[AnyMessage], add_messages]

In [35]:
template = '''
You are a highly intelligent and interactive assistant that uses three tools—`generate_scenario`, `review_speech`, and —to provide users with a seamless and engaging experience.

The following variables have been provided:
- **Username**: A string representing the user's name.
- **user_id**: A unique identifier for the user, used as an argument in the generate_scenario tool.
- **Incomplete Tasks**: A string containing the details of an incomplete task, or `NULL` if no tasks are pending.

user_id = UserName:tharun, id:1, Incmplete Tasks: NULL

Use the username to make your responses engaging where appropriate but avoid overusing it. Respond dynamically based on the provided `Incomplete Tasks` variable.

### Tools Overview:
1. **generate_scenario**:
   - **Input**: `user_id` (int)
   - Generate a new public speaking task for the user.
   - Save the task as incomplete for the user.

2. **review_speech**:
   - **Input**: `scenario` (string)
   - Review a video provided by the user for a specific task scenario.
   - Provide actionable feedback and strict scores for Content, Pronunciation, and Facial Expressions.
   - Store the scores in the database.

### Instructions:
1. **Handle Incomplete Tasks**:
   - If the `Incomplete Tasks` variable is `NULL`:
     - Use the `generate_scenario` tool to create a new task.
     - Construct a response: "Here's your new task: (generated_task)."
     - Use `final_answer` to send this response to the user.
   - If the `Incomplete Tasks` variable is not `NULL`:
     - Construct an engaging and concise response explaining the scenario, including all key points from the provided task details.
     - Example: "You have an unfinished task: 'Deliver a speech on the importance of mental health.' Please complete it before starting a new one."

2. **Generate New Task**:
   - When prompted by the user to start a new task, use the `generate_scenario` tool.
   - Construct a response with the generated task description and save it as incomplete for the user.
   - Example: "Here's your new task: (generated_task)."

3. **Review Speech**:
   - If the user provides a video, use the `review_speech` tool.
   - Construct the response with:
     - Detailed feedback for Content, Pronunciation, and Facial Expressions.
     - Strict scores for each category.
     - Acknowledgment that the scores have been saved.

4. **General Behavior**:
   - Be engaging, clear, and helpful in your responses.
   - Confirm user actions before proceeding where necessary.
   - Handle errors gracefully and provide guidance for retrying if needed.

### Example Scenarios:

1. **Incomplete Task Provided**:
   - **Variable `Incomplete Tasks`**: 'Speak about the importance of mental health in a community seminar.'
   - **Assistant**:
     Construct the response: "You have an unfinished task: 'Speak about the importance of mental health in a community seminar.' Please complete it before starting a new one."

2. **No Incomplete Task**:
   - **Variable `Incomplete Tasks`**: `NULL`
   - **Assistant**:
     Use the `generate_scenario` tool to create a new task.
     Construct the response: "Here's your new task: 'Deliver a speech on the impact of technology in education at a university lecture.'"

3. **Video Review**:
   - **Assistant**:
     Construct the response:
       ```
       Great job! Here’s your feedback:
       - **Content**: Your speech was well-structured and engaging. However, consider adding more examples to clarify your points.
       - **Pronunciation**: Clear and fluent for the most part, but some words were slightly rushed—practice pacing.
       - **Facial Expressions**: Good use of expressions to emphasize points. Could smile more to connect better with the audience.
       - Scores:
         - Content: 8/10
         - Pronunciation: 7/10
         - Facial Expressions: 8/10
       ```
       "Your scores have been saved. Keep up the great work!"

4. **Error Handling**:
   - **Assistant**:
     Construct the response: "Oops, something went wrong. Please try again after some time."

Stay interactive, engaging, and proactive in assisting the user with their public speaking journey!
'''


In [36]:
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import Runnable, RunnableConfig


class Assistant:
    def __init__(self, runnable: Runnable):
        self.runnable = runnable

    def __call__(self, state: State, config: RunnableConfig):
        while True:
            configuration = config.get("configurable", {})
            user_id = configuration.get("user_id", None)
            state = {**state, "user_info": user_id}
            result = self.runnable.invoke(state)
            if not result.tool_calls and (
                not result.content
                or isinstance(result.content, list)
                and not result.content[0].get("text")
            ):
                messages = state["messages"] + [("user", "Respond with a real output.")]
                state = {**state, "messages": messages}
            else:
                break
        return {"messages": result}



prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            template,
        ),
        ("placeholder", "{messages}"),
    ]
)

tools = [
    generate_scenario,
    review_speech
    
]
part_1_assistant_runnable = prompt | llm.bind_tools(tools)

In [8]:
connection = psycopg2.connect(**db_config)

cursor = connection.cursor()

find_query = """
    SELECT * 
    FROM users
    WHERE email = %s;
    """
cursor.execute(find_query, ("dummyuser@example.com",))
user_details = cursor.fetchone()
user_details

(1, 'dummyuser', 'dummyuser@example.com', 'securepassword123', None)

In [37]:
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import END, StateGraph, START
from langgraph.prebuilt import tools_condition

builder = StateGraph(State)


# Define nodes: these do the work
builder.add_node("assistant", Assistant(part_1_assistant_runnable))
builder.add_node("tools", create_tool_node_with_fallback(tools))
# Define edges: these determine how the control flow moves
builder.add_edge(START, "assistant")
builder.add_conditional_edges(
    "assistant",
    tools_condition,
)
builder.add_edge("tools", "assistant")

# The checkpointer lets the graph persist its state
# this is a complete memory for the entire graph.
memory = MemorySaver()
graph = builder.compile(checkpointer=memory)

In [38]:
config = {
    "configurable": {
        # The passenger_id is used in our flight tools to
        # fetch the user's flight information
        "user_id": "UserName : Tharun , user_id : 123, Incomplete Tasks : NULL",
        # Checkpoints are accessed by thread_id
        "thread_id": 123,
    }
}

In [ ]:
out = graph.invoke(
    {"messages": ("user", "Hi")},
      config
)

In [ ]:
out.get("messages")[-1].content

In [11]:
from langchain_groq import ChatGroq
import os
from langchain import PromptTemplate
import psycopg2
from psycopg2 import sql
from langchain.tools import tool
from langchain_core.messages import ToolMessage
from langchain_core.runnables import RunnableLambda
from langgraph.prebuilt import ToolNode
from langchain_community.tools.tavily_search import TavilySearchResults
from typing import Annotated
from typing_extensions import TypedDict
from langgraph.graph.message import AnyMessage, add_messages
from langchain.memory import ConversationBufferMemory
from langchain.chains import LLMChain
from langchain_core.prompts import PromptTemplate

In [44]:
def generate_scenario(user_id,interest):
    """
    Generates a thought-provoking but relatable scenario where the user is speaking at a public event
    and stores it in the user's tasks table as an uncompleted task. The task is associated with the user_id, an integer.

    Args:
        user_id (int): The user_id of the user for whom the scenario is generated and stored.
        interest (str): The field in which the user expects the genrated scenario to be 
    
    Returns:
        str: A detailed scenario for public speaking based on the user's preferences.
    """
    db_config = {
    "user": os.getenv("DB_USER"),
    "password": os.getenv("DB_PASSWORD"),
    "host": os.getenv("DB_HOST"),
    "port": os.getenv("DB_PORT"),
    "dbname": os.getenv("DB_NAME"),
    }

    template = '''
    You are tasked with generating engaging, innovative, and easy-to-understand public speaking scenarios. Each scenario should provide the user with a unique opportunity to practice their speaking skills. Ensure the scenario is fresh and has not been generated for this user before, leveraging memory to avoid repetitions. Additionally, incorporate the user's specified field or genre of interest when provided. If no interest is specified (NULL), proceed with generating a general scenario as usual.
    ## User Interest : {interest}

    ### Guidelines for Scenario Generation:
    1. **Engagement and Clarity**:
    - The scenario must be intriguing and relatable to encourage user participation.
    - Keep the description crisp and clear, focusing only on essential details while maintaining creativity and interest.

    2. **Skill Development**:
    - Design scenarios that challenge the user in a constructive way, helping them improve aspects of public speaking such as confidence, clarity, storytelling, persuasion, or audience engagement.
    - The scenario should align with real-life speaking opportunities, such as professional, social, or creative settings.

    3. **Incorporating User Interests**:
    - If the user specifies a field or genre of interest (e.g., motivational speaking, professional presentations, storytelling):
        - Tailor the scenario to align with the specified interest.
        - Ensure the scenario remains engaging and relevant to the user’s goals.
    - If the user specifies "nothing specific" or the interest is NULL:
        - Explicitly proceed with generating a general scenario while adhering to the outlined guidelines.

    4. **Scenario Structure**:
    - **Title**: Provide a descriptive and creative title that captures the essence of the scenario.
    - **Setup**: Briefly describe the context or situation where the user will speak.
    - **Task**: Clearly outline the objective of the speech or presentation, emphasizing the speaking skills they will practice.

    5. **Examples of Scenarios**:
    - **Title**: "Inspiring Entrepreneurs: A Community Celebration"
        **Setup**: You have been invited to host a community event celebrating local entrepreneurs. The audience includes aspiring business owners, established entrepreneurs, and community members.
        **Task**: Your task is to deliver an opening speech that sets an inspiring and positive tone for the evening, highlighting the importance of entrepreneurship and community support.

    - **Title**: "Career Insights: Inspiring the Next Generation"
        **Setup**: Imagine you are addressing a group of high school students at their career day. They are eager to learn about various professions and future opportunities.
        **Task**: Share insights about your profession, narrate an impactful personal experience, and inspire them to explore their passions confidently.

    - **Title**: "The Big Pitch: Securing Startup Funding"
        **Setup**: You are presenting your innovative startup idea to a group of investors at a professional conference. The stakes are high, and the room is filled with potential backers.
        **Task**: Clearly explain your concept, outline its unique value, and persuade the investors to provide funding, while handling potential questions with confidence.

    - **Title**: "A Toast to Love: Wedding Speech"
        **Setup**: You are the best man/maid of honor at a close friend’s wedding. The reception hall is packed with friends, family, and well-wishers.
        **Task**: Deliver a heartfelt and entertaining toast that celebrates the couple, shares a personal anecdote, and leaves a lasting impression on the guests.

    - **Title**: "The Climate Debate: A Public Forum"
        **Setup**: You are moderating a panel discussion on climate change at a community forum. The panel includes scientists, activists, and local policymakers.
        **Task**: Introduce the topic, engage the panelists with insightful questions, manage audience interactions, and summarize key takeaways at the end.

    6. **Memory Usage**:
    - Check previous scenarios generated for the user to avoid repetition.
    - Ensure the generated scenario adds value and novelty to the user's learning experience.
    - Do not generate scenarios that match the user's past responses.

    7. **Tone and Accessibility**:
    - Maintain a supportive and friendly tone in the scenario description.
    - Use language that is simple and easy to understand for users of all proficiency levels.

    

    ### Notes:
    - Previous Responses: {chat_history}
    - Strongly prioritize incorporating the user's interest if provided. If the interest is NULL, proceed with a general scenario while maintaining creativity and relevance.
    - Avoid generating scenarios that have already been given to the user.
'''

    prompt = PromptTemplate.from_template(template)
    connection = None
    cursor = None
    try:
        memory = ConversationBufferMemory(memory_key="chat_history")
        model = LLMChain(
        llm=llm,
        prompt=prompt,
        verbose=True,
        memory=memory
        )


        response = model({"interest":interest})
        scenario = response.get('text')
        connection = psycopg2.connect(**db_config)
        cursor = connection.cursor()
        # Insert the scenario into the user's tasks table
        insert_query = sql.SQL(
            "INSERT INTO tasks (user_id, task, completed, score) VALUES (%s, %s, %s, %s)"
        )
        cursor.execute(insert_query, (user_id, scenario, False, None))
        connection.commit()
        return f"Scenario successfully generated {response['text']}"

    except psycopg2.Error as e:
        return f"Database error: {e}"
    except Exception as e:
        return f"Error generating scenario: {e}"
    finally:
        if cursor:
            cursor.close()
        if connection:
            connection.close()

In [45]:
resp = generate_scenario(1,'NULL')
resp



> Entering new LLMChain chain...
Prompt after formatting:

    You are tasked with generating engaging, innovative, and easy-to-understand public speaking scenarios. Each scenario should provide the user with a unique opportunity to practice their speaking skills. Ensure the scenario is fresh and has not been generated for this user before, leveraging memory to avoid repetitions. Additionally, incorporate the user's specified field or genre of interest when provided. If no interest is specified (NULL), proceed with generating a general scenario as usual.
    ## User Interest : NULL

    ### Guidelines for Scenario Generation:
    1. **Engagement and Clarity**:
    - The scenario must be intriguing and relatable to encourage user participation.
    - Keep the description crisp and clear, focusing only on essential details while maintaining creativity and interest.

    2. **Skill Development**:
    - Design scenarios that challenge the user in a constructive way, helping them improve 

'Scenario successfully generated ### Scenario Generation for Public Speaking Practice\n\nGiven that the user\'s interest is NULL, a general scenario will be created to provide an engaging and challenging public speaking opportunity.\n\n#### Title: "Community Champions: Recognizing Local Heroes"\n\n#### Setup: \nYou have been asked to host a community awards ceremony where local individuals who have made significant contributions to the community are being recognized. The event is attended by the awardees, their families, local officials, and members of the community.\n\n#### Task: \nYour task is to deliver a speech that warmly welcomes everyone to the ceremony, explains the significance of the awards, and introduces the first awardee. You will need to engage the audience by sharing a brief story of community service, highlight the importance of volunteerism, and keep the tone inspirational and celebratory. After introducing the first awardee, you will moderate a brief Q&A session, ensu

In [53]:
import google.generativeai as genai
import time
from langchain import PromptTemplate
import os
from dotenv import load_dotenv
import json
load_dotenv()
GOOGLE_API_KEY=os.getenv('GOOGLE_API_KEY')
import psycopg2
import re
genai.configure(api_key=GOOGLE_API_KEY)
from psycopg2 import sql

In [69]:
def parse_json_string(input_string):
    # Remove ```json or ``` if they exist
    cleaned_string = re.sub(r"```(?:json)?", "", input_string).strip()
    try:
        # Parse the cleaned string into JSON
        parsed_json = json.loads(cleaned_string)
        return parsed_json
    except json.JSONDecodeError as e:
        print(f"Error decoding JSON: {e}")
        return None

In [81]:
def score(video_url,task,task_id):
    db_config = {
    "user": os.getenv("DB_USER"),
    "password": os.getenv("DB_PASSWORD"),
    "host": os.getenv("DB_HOST"),
    "port": os.getenv("DB_PORT"),
    "dbname": os.getenv("DB_NAME"),
    }
    model = genai.GenerativeModel(model_name="gemini-1.5-pro")
    prompt = f'''
    You are an advanced scoring AI tool designed to evaluate user-submitted videos of public speaking tasks. Your role is to analyze the video and its corresponding task, providing constructive feedback and scores to help users improve their public speaking skills. Your response must include only the evaluation output in the specified JSON format and exclude any code block syntax or additional text.

    ### Evaluation Guidelines:

    1. **Feedback Description**:
      - Provide a detailed, 2000-character feedback description in paragraph format.
      - This feedback should:
        - Explain areas where the user performed well and areas needing improvement.
        - Offer actionable advice on how the user can enhance their public speaking skills in alignment with the task.
        - Focus on key aspects of public speaking, such as structure, clarity, delivery, engagement, and alignment with the task.
        - Maintain a constructive, encouraging, and friendly tone.

    2. **Scoring Criteria**:
      - Evaluate the user's performance across three key areas, each scored on a scale from 0 to 100:
        - **CONTENT**:
          - Assess how well the speech content aligns with the given task.
          - Consider factors such as relevance, structure, depth, and creativity of the content.
        - **FLUENCY**:
          - Evaluate the user's speech fluency, including smoothness, pace, and lack of interruptions or filler words.
        - **EXPRESSION**:
          - Analyze the user's body language, gestures, facial expressions, and emotional engagement with the audience.

    3. **Output Format**:
      - Provide the evaluation strictly in this format:
        {{
          "feedback_description": "2000-character long feedback in paragraph format.",
          "scores": {{
            "content": 0-100,
            "fluency": 0-100,
            "expression": 0-100
          }}
        }}

    4. **Unacceptable Format**:
      - Do NOT include any code block syntax like ` ```json ` or any enclosing text outside the JSON.
      - Example of unacceptable format:
        ```
        ```json
        {{
          "feedback_description": "Example feedback.",
          "scores": {{
            "content": 90,
            "fluency": 85,
            "expression": 80
          }}
        }}
        ```
        ```

    ### IMPORTANT:
    - Output only the JSON content as specified in the **Output Format** section.
    - Do not include any explanations, comments, or additional text outside the structured JSON output.

    CURRENT TASK: {task}
    '''


    connection = None
    cursor = None
    try:
      video_file = genai.upload_file(path=video_url)
      print(f"Completed upload: {video_file.uri}")
      while video_file.state.name == "PROCESSING":
        time.sleep(10)
        video_file = genai.get_file(video_file.name)

      if video_file.state.name == "FAILED":
        raise ValueError(video_file.state.name)
      response = model.generate_content([video_file,prompt],
                                    request_options={"timeout": 600})
      parsed_output = json.loads(response.text)
      connection = psycopg2.connect(**db_config)
      cursor = connection.cursor()

      update_query = sql.SQL("""
            UPDATE tasks
            SET score = %s, completed = TRUE
            WHERE task_id = %s
        """)
      cursor.execute(update_query, (json.dumps(parsed_output["scores"]), task_id))
      connection.commit()
      return parsed_output
    except psycopg2.Error as e:
       print(f'Databse Error: {e}')
    except Exception as e:
        return f"Error scoring: {e}"
    finally:
        if cursor:
            cursor.close()
        if connection:
            connection.close()

In [82]:
score("../user_vid/videoplayback (3).mp4","Deliver an inspirational speech to first-time marathon runners.",25)

Completed upload: https://generativelanguage.googleapis.com/v1beta/files/qxgiznitfkur
...Parsed:  {'feedback_description': "Chris, your speech on focus and its connection to a fulfilling life is insightful. However, it doesn't quite hit the mark for inspiring first-time marathon runners. While the concepts of scatterfocus and intentional disconnection are interesting, they lack direct relevance to the challenges and triumphs of marathon training and racing. Your personal anecdote about limiting phone use is relatable, but could be stronger if tied to athletic pursuits or overcoming obstacles.  Consider focusing your message on the mental resilience and focus required to run a marathon. Share stories of perseverance and overcoming challenges during training, the importance of setting goals, and the thrill of crossing the finish line.  Your delivery is energetic and engaging, but could benefit from a more structured approach. A clear beginning, middle, and end would make your message mor

{'feedback_description': "Chris, your speech on focus and its connection to a fulfilling life is insightful. However, it doesn't quite hit the mark for inspiring first-time marathon runners. While the concepts of scatterfocus and intentional disconnection are interesting, they lack direct relevance to the challenges and triumphs of marathon training and racing. Your personal anecdote about limiting phone use is relatable, but could be stronger if tied to athletic pursuits or overcoming obstacles.  Consider focusing your message on the mental resilience and focus required to run a marathon. Share stories of perseverance and overcoming challenges during training, the importance of setting goals, and the thrill of crossing the finish line.  Your delivery is energetic and engaging, but could benefit from a more structured approach. A clear beginning, middle, and end would make your message more impactful.  While the knitting example adds a personal touch, it might distract from the core me